In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("elt-pyspark-demo").getOrCreate()
path_file = '/home/workspaces/pyspark/etl-pyspark-demo/datalake/Cortex_04_03_2024-05_05_2024_historical_data_coinmarketcap.csv'

24/05/22 18:52:32 WARN Utils: Your hostname, DESKTOP-TE9D6P7 resolves to a loopback address: 127.0.1.1; using 172.31.179.110 instead (on interface eth0)
24/05/22 18:52:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/22 18:52:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
df = spark.read.csv(
    path_file, 
    header=True, 
    sep=';'
)

In [7]:
df.head()

Row(timeOpen='2024-05-04T00:00:00.000Z', timeClose='2024-05-04T23:59:59.999Z', timeHigh='2024-05-04T23:31:00.000Z', timeLow='2024-05-04T10:00:00.000Z', name='2781', open='0.3104479022', high='0.3331378439', low='0.3076670714', close='0.3281821223', volume='4827456.02', marketCap='72394383.56', timestamp='2024-05-04T23:59:59.999Z')

### RDD – Resilient Distributed Datasets

#### Processos de transformação e ações

In [16]:
sc = spark.sparkContext
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

In [17]:
type(rdd)

pyspark.rdd.RDD

In [20]:
# filtro feito com função lambda sobre um RDD
rdd.filter(lambda f: f == 2).collect()

[2]

In [24]:
# aplicando multiplicação com a função map sobre um RDD
rdd.map(lambda f: f * 2).collect()

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

In [28]:
# unido dois RDD
soma = rdd.map(lambda f: f * 2)
rdd.union(soma).collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

In [29]:
# procurando uma interseção em um RDD
soma = rdd.map(lambda f: f * 2)
rdd.intersection(soma).collect()

[2, 4, 6, 8, 10]

## Dataframes

In [17]:
from pyspark.sql import functions as Func
#spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
abertura = df.withColumn("data", Func.to_timestamp(Func.col("timeOpen"), "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"))
# conta quantas vezes o ano se repete durante esse dataframe
abertura.select("data").groupBy(Func.year("data")).count().show()
#abertura.head()

+----------+-----+
|year(data)|count|
+----------+-----+
|      2024|   60|
+----------+-----+



In [35]:
#preco = df.withColumn("cotacao", Func.col("open"))
#preco.select("cotacao").groupBy("cotacao").count().show()
df.select("open", "timeOpen").orderBy("timeOpen").show()

+------------+--------------------+
|        open|            timeOpen|
+------------+--------------------+
|0.5373953256|2024-03-06T00:00:...|
|0.5982707251|2024-03-07T00:00:...|
|0.6220453242|2024-03-08T00:00:...|
|0.6143146033|2024-03-09T00:00:...|
|0.8150565578|2024-03-10T00:00:...|
|0.6960503913|2024-03-11T00:00:...|
| 0.628751318|2024-03-12T00:00:...|
|0.5581168596|2024-03-13T00:00:...|
|0.5224376319|2024-03-14T00:00:...|
|0.5148657252|2024-03-15T00:00:...|
| 0.497326043|2024-03-16T00:00:...|
|0.4364030983|2024-03-17T00:00:...|
| 0.494266342|2024-03-18T00:00:...|
|0.4595448809|2024-03-19T00:00:...|
| 0.389506345|2024-03-20T00:00:...|
|0.4317766705|2024-03-21T00:00:...|
| 0.411620251|2024-03-22T00:00:...|
|0.4095128857|2024-03-23T00:00:...|
|0.4173736789|2024-03-24T00:00:...|
| 0.431016679|2024-03-25T00:00:...|
+------------+--------------------+
only showing top 20 rows

